## Mutual Information Feature Selection

The aim of this notebook:
- Look at column features and how they influence a cell's self-exciting property (mutual info)

# Why conditional mutual information is larger than mutual information
The conditional variable has more possible states and therfore the join has a higher entropy - the maximum value it can take on can therefore be higher - to make plots easier to read we can normalize the mutual information with the mutual information of the target signal with itself.

$I(X;Y) = H(X) + H(Y) - H(X,Y)$


$I(X;Y|Z) = H(X,Z) + H(Y,Z) - H(X,Y,Z) - H(Z)$

# Interactive Plotly Plots

In [1]:
from sparse_discrete_table import SparseDiscreteTable, build_discrete_table
from ipywidgets import widgets
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from utils.data_processing import time_series_to_time_index
from pprint import pprint

In [2]:
from sparse_discrete_table import quick_mutual_info, quick_cond_mutual_info
from sparse_discrete_table import mutual_info_over_time, conditional_mutual_info_over_time
from utils.interactive import filter_frame, get_total_counts, new_bins, new_int_bins
from utils.interactive import get_mean_map, bin_data_frame, State

In [3]:
pd.set_option('mode.chained_assignment', None)

`Just for reference where Crimes_Chicago_2001_to_2019_datetime_fmt came from:`
```python
# was used to convert original csv to dataframe and string dates to datetime format
df = pd.read_csv("data/original/Crimes_Chicago_2001_to_2019.csv")
df['Date'] = pd.DatetimeIndex(pd.to_datetime(df['Date']))
df = df[[
    'ID',
    'Date',
    'Primary Type',
    'Arrest',
    'Latitude',
    'Longitude',
]].dropna()
df.to_pickle("data/raw/Crimes_Chicago_2001_to_2019_datetime_fmt.pkl") # Dates are type datetime not string
```

In [4]:
df = pd.read_pickle("data/raw/Crimes_Chicago_2001_to_2019_datetime_fmt.pkl")

In [5]:
from pandas.api.types import CategoricalDtype


crime_categories = [
        "THEFT",
        "BATTERY",
        "CRIMINAL DAMAGE",
        "NARCOTICS",
        "ASSAULT",
        "BURGLARY",
        "MOTOR VEHICLE THEFT",
        "ROBBERY",
]


# Label encode the crime catgegories - makes histograms faster
CrimeType = CategoricalDtype(categories=crime_categories, ordered=True)
df['Primary Type'] = df['Primary Type'].astype(CrimeType)

category_mask = df['Primary Type'].isin(crime_categories)

In [6]:
# used to remove the anomalies
LAT_MIN, LAT_MAX = 41.6445, 42.0228

LON_MIN, LON_MAX = -87.9344, -87.5244

FREQ = '1D' # '1H' #

if FREQ == '1D':
    DATE_MIN, DATE_MAX = '2012-01-01', '2019-01-01'
else:
    DATE_MIN, DATE_MAX = '2012-01-01', '2012-10-01'


lon_mask = (df.Longitude > LON_MIN) & (df.Longitude < LON_MAX)
lat_mask = (df.Latitude > LAT_MIN) & (df.Latitude < LAT_MAX)
date_mask = (df.Date >= DATE_MIN) & (df.Date < DATE_MAX)

In [7]:
df = df[lon_mask & lat_mask & date_mask & category_mask]

In [8]:
df['c'] = df['Primary Type'].cat.codes

In [9]:
df['t'] = time_series_to_time_index(t_series=df.Date, t_step=FREQ, floor=True)

In [10]:
# date selectors
date_range = pd.date_range(df.Date.min().ceil(FREQ), df.Date.max().floor(FREQ), freq=FREQ)

In [11]:
from geopy import distance
from utils.utils import ffloor, fceil

# get meter per degree estimat
coord_series = df[['Latitude', 'Longitude']]

lat_min, lon_min = coord_series.min()
lat_max, lon_max = coord_series.max()

lat_mean, lon_mean = coord_series.mean()

dy = distance.distance((lat_min, lon_min), (lat_max, lon_min)).m
dx = distance.distance((lat_min, lon_min), (lat_min, lon_max)).m

lat_per_metre = (lat_max - lat_min)/dy
lon_per_metre = (lon_max - lon_min)/dx

ratio_xy = dx/dy
print(f"ratio_xy: {ratio_xy}")
print(f"lat_per_metre: {lat_per_metre}")
print(f"lon_per_metre: {lon_per_metre}")

ratio_xy: 0.8127895963778615
lat_per_metre: 9.003326781922841e-06
lon_per_metre: 1.2003344797182708e-05


### Very important  - all the meta data is mapped from intervals of 0.001 in the lat and lon space with ratios of 8 and 11 to ensure that the grids cels are square

#### Confirm the make_grid technique and hist product the same output

In [12]:
# [20,16] - [10,8] - [5,4] - [3,2] - [1,1]
x_scale, y_scale = 10,8 #3,2
xy_scale = np.array([x_scale, y_scale])  # must be integer so that we can easily sample demographic data
dlon, dlat = xy_scale * np.array([0.001, 0.001])
meta_info = {}
meta_info["x_scale"] = x_scale
meta_info["y_scale"] = y_scale
meta_info["dlon"] = float(dlon)
meta_info["dlat"] = float(dlat)
meta_info["x in metres"] = 85000 * float(dlon)
meta_info["y in metres"] = 110000 * float(dlat)

pprint(meta_info)

# use perfect squares perfect squares
"""
all the meta data is mapped from intervals of 0.001 in 
the lat and lon space with ratios of 8 and 11 to ensure that the grids cels are square
"""
# cell_size_m = 430
# dlat = cell_size_m*lat_per_metre
# dlon = cell_size_m*lon_per_metre
 
# use increments of 0.001

{'dlat': 0.008,
 'dlon': 0.01,
 'x in metres': 850.0,
 'x_scale': 10,
 'y in metres': 880.0,
 'y_scale': 8}


'\nall the meta data is mapped from intervals of 0.001 in \nthe lat and lon space with ratios of 8 and 11 to ensure that the grids cels are square\n'

In [13]:
from pprint import pformat        
        
def default_state():
    return  State(
        dlon=dlon,
        dlat=dlat,
        lon_max = LON_MAX,
        lon_min = LON_MIN,
        lat_max = LAT_MAX,
        lat_min = LAT_MIN,     
        total_lon_max = LON_MAX,
        total_lon_min = LON_MIN,
        total_lat_max = LAT_MAX,
        total_lat_min = LAT_MIN,         
        date_min = DATE_MIN,
        date_max = DATE_MAX,                
        time_index = 0,
        date_indices = (0,len(date_range)-1),
        date_range = pd.date_range(df.Date.min().ceil(FREQ), df.Date.max().floor(FREQ), freq=FREQ),
        freq=FREQ,
        crime_types = [*crime_categories],    
        mi_max_offset = 35,
        block_size_index=2,    
        conditional_temporal_variables= ["Day of Week"],# ["Hour", "Day of Week", "Time of Month", "Time of Year"],
    )
    
state = default_state()
state

{'block_size_index': 2,
 'callbacks': [],
 'conditional_temporal_variables': ['Day of Week'],
 'crime_types': ['THEFT',
                 'BATTERY',
                 'CRIMINAL DAMAGE',
                 'NARCOTICS',
                 'ASSAULT',
                 'BURGLARY',
                 'MOTOR VEHICLE THEFT',
                 'ROBBERY'],
 'date_indices': (0, 2556),
 'date_max': '2019-01-01',
 'date_min': '2012-01-01',
 'date_range': DatetimeIndex(['2012-01-01', '2012-01-02', '2012-01-03', '2012-01-04',
               '2012-01-05', '2012-01-06', '2012-01-07', '2012-01-08',
               '2012-01-09', '2012-01-10',
               ...
               '2018-12-22', '2018-12-23', '2018-12-24', '2018-12-25',
               '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29',
               '2018-12-30', '2018-12-31'],
              dtype='datetime64[ns]', length=2557, freq='D'),
 'dlat': 0.008,
 'dlon': 0.01,
 'freq': '1D',
 'lat_max': 42.0228,
 'lat_min': 41.6445,
 'lon_max': -87.5244,
 

In [14]:
tbins, cbins, ybins, xbins = new_bins(df, state)

`Reference for code to compare grids with previous generation techniques:`
```python
file_location = "data/processed/T24H-X425M-Y440M_2013-01-01_2017-01-01/"

generated_data = np.load(file_location + "generated_data.npz")
x_range = generated_data['x_range']
y_range = generated_data['y_range']
t_range = pd.read_pickle(file_location + "t_range.pkl")

crime_types_grids = generated_data['crime_types_grids'] 

xbins  = x_range - dlon/2
xbins = np.array([*xbins,xbins[-1]+dlon])

ybins  = y_range - dlat/2
ybins = np.array([*ybins,ybins[-1]+dlat])

import json

with open(file_location + "info.json", "r") as f:
    meta_info = json.load(f)
         
meta_info
```

### Widget Plot

In [15]:
# reset state
state.crime_types = [*crime_categories]

state.date_indices = (0,len(date_range)-1)
state.date_min, state.date_max = date_range[[*state.date_indices]]

state.date_range = pd.date_range(
    state.date_min.ceil(state.freq), 
    state.date_max.floor(state.freq), 
    freq=state.freq, 
)

In [16]:
font_dict = dict(
    family="Time New Roman",     
    size=14,
#     color="RebeccaPurple",
)

# quick plotly plot builders
def new_scatter(title, ylabel, xlabel, font_dict=None):
    return go.FigureWidget(
        data=go.Scatter(),
        layout=dict(
            title_text=title,
            title_x=0.5,        
            xaxis_title=xlabel,
            yaxis_title=ylabel,
            legend_title="Legend Title",
            font=font_dict,
        ),
    )


def new_heatmap(z):
    return go.Figure(
        data=go.Heatmap(z=z),
        layout=dict(
            height=600,
#             width=ratio_xy*fig_height,
            yaxis=dict(scaleanchor="x", scaleratio=1/ratio_xy),
        ),    
    )

In [17]:
fig_total_counts = new_scatter(title="Total Crimes",ylabel="Total Counts",xlabel="Date Time")

In [18]:
# MI Plots on a single curve
fig_mi_cmi = go.FigureWidget(
        data=[
            go.Scatter(
                name="$I(C_t;C_{t-k})$",#"Mutual Information",
            ),
            go.Scatter(
                name="$I(C_t;C_{t-k}|D_t;D_{t-k})$", #"Condtional Mutual Information",
            ),
        ],
        layout=dict(
            title_text="Mutual and Condtional Mutual Information",
            title_x=0.5,        
            xaxis_title="Offest in days (k)",
            yaxis_title="Normalised Score [0,1]",
            legend_title="Curves",
#             font=font_dict,
        ),
    )

line_mi, line_cmi = fig_mi_cmi.data

In [19]:
counts_mean = get_mean_map(df, state)

In [20]:
scattergl = go.Scattergl(
    hoverinfo='skip',
    mode='markers',
    name='scattergl',
    marker_color='red',
)

heatmapgl = go.Heatmap(
    z=counts_mean,
    x=xbins,
    y=ybins, 
    hoverinfo='skip',
    colorscale='viridis',
    opacity=1,
    name='heatmapgl',
)


fig_height = 600
fig_app = go.FigureWidget(
    data=[heatmapgl, scattergl],
    layout=dict(
        margin={"r":50,"t":30,"l":0,"b":0},
        height=fig_height,
#         width=fig_height*ratio_xy,
        yaxis=dict(scaleanchor="x", scaleratio=1/ratio_xy),
        clickmode='event+select',
    ),
)

scattergl = fig_app.data[1]
heatmapgl = fig_app.data[0]

#### Update figures based on state

In [21]:
block_size_changed = True

In [22]:
def construct_temporal_information(data_frame, state, month_divisions=3, year_divisions=24):
    sub_dt = date_range[data_frame.t.min():data_frame.t.max()+1]

    conds_dict = dict()
    
    if "Hour" in state.conditional_temporal_variables:
        conds_dict["Hour"] = sub_dt.hour
                
    if "Day of Week" in state.conditional_temporal_variables:
        conds_dict["Day of Week"] = sub_dt.dayofweek

    if "Time of Month" in state.conditional_temporal_variables:
        conds_dict["Time of Month"] = np.floor((month_divisions-1e-8)*sub_dt.day/sub_dt.days_in_month)

    if "Time of Year" in state.conditional_temporal_variables:
        conds_dict["Time of Year"] = np.floor((year_divisions-1e-8)*sub_dt.dayofyear/(365 + sub_dt.is_leap_year*1))
    
    conds = pd.DataFrame(conds_dict)
    
    return conds

In [29]:
if FREQ == '1D':
    cycles = 7,
elif FREQ == '1H':
    cycles = 7,7*24
else:
    raise Exception(f"FREQ={FREQ} is not a valid value: 24H or 1D")

log_norm = True

# global updater
def draw(state):  
    global block_size_changed
    sub = filter_frame(df, state)
    total_counts_y, total_counts_x = get_total_counts(sub, state, date_range)
    
    cycles = 1,
    conds = construct_temporal_information(
        data_frame=sub,
        state=state,
        month_divisions=3,
        year_divisions=24,
    )
    
    mi_y, mi_x = mutual_info_over_time(
        a=total_counts_y, 
        max_offset=state.mi_max_offset,
        norm=False,#True, 
        log_norm=log_norm,
        include_self=False,
    )
    cmi_y, cmi_x = conditional_mutual_info_over_time(
        a=total_counts_y,
        max_offset=state.mi_max_offset,
        norm=False,#True, 
        log_norm=log_norm,
        include_self=False,
        cycles=cycles,
        conds=conds,
    )
    
    slide = sub[sub.t == state.time_index]
    
    with fig_app.batch_update():
        if block_size_changed:
            mean_map, xbins, ybins = get_mean_map(df, state)
            block_size_changed = False
            
            heatmapgl.x = xbins
            heatmapgl.y = ybins
            heatmapgl.z = mean_map
        
        
        scattergl.x=slide.Longitude
        scattergl.y=slide.Latitude
        
        fig_total_counts.data[0].x = total_counts_x
        fig_total_counts.data[0].y = total_counts_y
        
        line_mi.x = mi_x
        line_mi.y = mi_y
        
        line_cmi.x = cmi_x
        line_cmi.y = cmi_y
        
        fig_mi_cmi.update_layout(
            title_text=f"Mutual and Condtional Mutual Information\ncondtioned on {state.conditional_temporal_variables}")

In [30]:
def scattergl_on_select(trace,points,selector):
    global lat_filter, lon_filter
    if trace.name == 'scattergl':                
        state.lon_min, state.lon_max = selector.xrange
        state.lat_min, state.lat_max = selector.yrange
        
        draw(state)
            
scattergl.on_selection(scattergl_on_select)

In [31]:
# widget setup
from ipywidgets import Layout, widgets

# helper functions
def get_widget_index(change):
    if isinstance(change, dict) and change.get('name') == 'index':
        return change.get('new')
    return None

def get_widget_value(change):
    if isinstance(change, dict) and change.get('name') == 'value':
        return change.get('new')
    return None

#### Widgets and how they should update state

In [32]:
# time index date display label
current_date_label = widgets.Label(f'Date: {date_range[state.time_index].strftime("%c")}')

# time index selector
time_index_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(date_range)-1,
    step=1,
    description='Time Index:',
    continuous_update=False,
    layout=Layout(width='80%'),
)

def on_change_time_index(change):
    global current_date_label, state
    time_index = get_widget_value(change)
    if time_index:
        current_date_label.value = f'Date: {date_range[time_index].strftime("%c")}'
        state.time_index = time_index
        draw(state)
        
time_index_slider.observe(on_change_time_index)

play_button = widgets.Play(
    value=0,
    min=time_index_slider.min,
    max=time_index_slider.max,
    step=1,
    interval=800,
    description="Press play",
    disabled=False
)
widgets.jslink((play_button, 'value'), (time_index_slider, 'value'))


# Date Slider
date_range_slider = widgets.SelectionRangeSlider(
    options=[d.strftime('%y/%m/%d') for d in date_range],
    index=(0, len(date_range)-1),
    description='Date Range:',
    disabled=False,
    orientation = 'horizontal',
    layout=Layout(width='95%'),
    continuous_update=False,
)

def on_change_date_range_slider(change):
    global state, time_index_slider
    
    index = get_widget_index(change)
    if index:        
        i, j = index
        
        time_index_slider.value = i 
        time_index_slider.min = i
        time_index_slider.max = j
                
        state.update(
            date_range_indices=(i,j),  
            date_min=date_range[i],
            date_max=date_range[j],        
        )
        draw(state)

date_range_slider.observe(on_change_date_range_slider)


# crime type selector
def on_change_crime_types(button):
    global state
    if button.button_style == "":
        button.button_style = "success"
        state.crime_types.append(button.description)
    else:
        button.button_style = ""
        state.crime_types.remove(button.description)
    draw(state)
    
def new_buttons(names,all_selected=False):
    global state
    buttons = []
    
    if all_selected:
        state.crime_types = [*names]
        button_style = "success"
    else:
        state.crime_types = []
        button_style = ""
        
    for name in names:
        button = widgets.Button(
            description=name,
            tooltip=name,
            disabled=False,
            button_style=button_style, # 'success', 'info', 'warning', 'danger' or ''
        )
        
        button.on_click(on_change_crime_types)
        
        buttons.append(button)
    return buttons
    
    
time_selectors = widgets.VBox([
    widgets.HBox([date_range_slider]),
    widgets.HBox([time_index_slider])    
])    

buttons = new_buttons(names=crime_categories, all_selected=True)
crime_selectors = widgets.VBox([
    widgets.HBox(buttons[:4]),
    widgets.HBox(buttons[4:]),
])

# Spatial block size selection widget
dlon_dlat_opts = list(map(tuple,(np.array([[40,32], [20,16],[10,8], [5,4], [3,2], [1,1]]) * np.array([0.001, 0.001]))))
dx_dly_opts = list(map(tuple,(np.array([[40,32], [20,16],[10,8], [5,4], [3,2], [1,1]]) * np.array([85, 110]))))

dropdown_block_size = widgets.Dropdown(
    options=[(v, i) for i,v in enumerate(dx_dly_opts)],
    value=state.block_size_index,
    description='dx, dy:',
    disabled=False,
)

def on_change_dropdown_block_size(change):
    global state, block_size_changed
    value = get_widget_value(change)
    if value is not None:  
        state.block_size_index = value
        state.dlon, state.dlat = dlon_dlat_opts[value]
        block_size_changed = True
        draw(state)
        
dropdown_block_size.observe(on_change_dropdown_block_size)


# widgets combinations and setup
row0 = widgets.HBox([play_button, crime_selectors, dropdown_block_size])
row1 = time_selectors

controller = widgets.VBox([row0, row1, current_date_label])

In [33]:
# state.conditional_temporal_variables = ["Hour", "Day of Week", "Time of Month", "Time of Year"]
state.conditional_temporal_variables = [ "Day of Week" ] #["Day of Week", "Time of Month", "Time of Year"]

In [34]:
state.mi_max_offset = 400
draw(state)
widgets.VBox([fig_mi_cmi, fig_total_counts, controller, fig_app])

    'data': [{'name': '$I(C_t;C_{t-k})$',
              'type': 'scatter',
     …

In [36]:
from utils.interactive import state_to_conf
state_to_conf(state)

{'start_date': '2012-01-01',
 'end_date': '2019-01-01',
 'dT': '1D',
 'x_scale': 10,
 'y_scale': 8,
 'lat_max': 41.981,
 'lat_min': 41.835,
 'lon_max': -87.609,
 'lon_min': -87.801,
 'root': './',
 'crime_types': ['THEFT',
  'BATTERY',
  'CRIMINAL DAMAGE',
  'NARCOTICS',
  'ASSAULT',
  'BURGLARY',
  'MOTOR VEHICLE THEFT',
  'ROBBERY']}

### Save selected subset limits to json file to be generated at later stage

In [37]:
from datetime import datetime
from utils.interactive import state_to_conf
from utils.utils import write_json

SAVE_SELECTION = False
if SAVE_SELECTION:
    conf = state_to_conf(state)
#     ref = datetime.now().isoformat().split('.')[0]
    ref = hex((hash(conf.__repr__()) % 3000) + 300).split('x')[1]
    
    print(f"Saving file with ref: {ref}")
    write_json(
        data=conf,
        file_name=f"./config/generate_data_{ref}.json",
    )

Saving file with ref: c52


### Statistics on selection

In [ ]:
sub = filter_frame(df, state)

binned_data, bins = bin_data_frame(sub, state)

sub_dt = date_range[sub.t.min():sub.t.max()+1]
counts_sum = binned_data.sum(-1).sum(-1)

print(len(counts_sum),len(sub_dt))
assert len(counts_sum) == len(sub_dt)


data_dict = {k: counts_sum[:,i]   for i,k in enumerate(state.crime_types)}
# data_dict['Date'] = sub_dt
# data_dict['Hour'] = sub_dt.hour
# data_dict['Weekday'] = sub_dt.dayofweek

tota_counts_by_type = pd.DataFrame(data_dict)
tota_counts_by_type

In [ ]:

# counts_sum = np.round(np.log2(1 + counts_sum))

# cap = 33
# counts_sum[counts_sum > cap] = cap

traces = []
for i,k in enumerate(state.crime_types):
    traces.append(
#         go.Scatter(
#             y=counts_sum[:,i],
#             name=k,
#         ),
        go.Histogram(
            x=counts_sum[:,i],
            name=k,
            opacity=0.4,
            histnorm='probability density',
        ),
#         go.Violin(
#             y=counts_sum[:,i],
#             name=k,
#             opacity=0.4,
#         ),   
    )
    
fig = go.Figure(traces, layout=dict(height=600))
fig.update_layout(barmode='overlay')
fig.show()

In [ ]:
# distribution of selection counts


fig = px.histogram(
    tota_counts_by_type.melt(),
    histnorm='probability density',
    title="Crime Count Distribution",
#     color="variable",
    facet_col="variable",
    facet_col_wrap=1,
    opacity=.3,
)

fig.update_layout(barmode='overlay', height=1200, width=500)
fig.show()

In [ ]:
# distribution of selection counts
total_counts_y, total_counts_x = get_total_counts(sub, state, date_range)
fig = px.histogram(pd.DataFrame({"Crime Count":total_counts_y}),
             histnorm='probability density',title="Crime Count Distribution")
fig.update_layout(title={'x':0.5})
fig.update_yaxes(title_text='Probability')
fig.update_xaxes(title_text='Crime Count')
fig.show()

In [ ]:
sub_dow_name = pd.DataFrame({'day of week name': sub.Date.dt.day_name()})
sub_dow = pd.DataFrame({'day of week': sub.Date.dt.dayofweek})
sub_hour = pd.DataFrame({'hour of day': sub.Date.dt.hour})
sub_cat = pd.concat([sub, sub_dow, sub_dow_name, sub_hour], axis=1)
sub_cat['crimes'] = np.ones(len(sub))


# distribution of selection counts
total_counts_y, total_counts_x = get_total_counts(sub, state, date_range)

fig = px.histogram(pd.DataFrame({"Crime Counts":total_counts_y}),
             histnorm='probability density',title="Crime Count Distribution")
fig.update_yaxes(title_text='Probability')
fig.update_xaxes(title_text='Crime Count')
fig.update_layout(title={'x':0.5})
fig.show()

# log normed distribution
log_scaled_counts_y = np.round(np.log2(1+ total_counts_y))
fig = px.histogram(pd.DataFrame({"Log Scaled Crime Counts":log_scaled_counts_y}),
             histnorm='probability density',title="Log Crime Count Distribution log2(1+x)")
fig.update_yaxes(title_text='Probability')
fig.update_xaxes(title_text='Log Crime Count')
fig.update_layout(title={'x':0.5})
fig.show()

# distribution of crime type
fig = px.histogram(
    data_frame=sub_cat,
    x='Primary Type',
    y='crimes', 
    histnorm='probability density',
    title="Crime Type Distribution",
)
fig.update_layout(title={'x':0.5})
fig.update_yaxes(title_text='Probability')
fig.update_xaxes(title_text='Crime Count')
fig.show()


# crime distirbution distribution of hour of day by type
fig = px.histogram(
    data_frame=sub_cat,
    x='hour of day',
    y='crimes', 
    facet_col='Primary Type',
    facet_col_wrap=2,
    height=800,
    histnorm='probability density',
    title="Crime Type Distribution by Hour",
)
fig.update_layout(title={'x':0.5})
fig.update_yaxes(title_text='Probability',col=1)
fig.update_xaxes(title_text='Hour of Day')
fig.show()

# distribution of day of week
fig = px.histogram(
    data_frame=sub_cat,
    x='day of week',
    y='crimes', 
    facet_col='Primary Type',
    facet_col_wrap=2,
    height=800,
    histnorm='probability density',
    title="Crime Type Distribution by Weekday",
)
fig.update_layout(title={'x':0.5})
fig.update_yaxes(title_text='Probability',col=1)
fig.update_xaxes(title_text='Day of Week')
fig.show()

In [ ]:
for i in enumerate(['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']):
    print(i)

In [ ]:
# binned_data, bins = bin_data_frame(df,state)
mean_map = counts_mean = get_mean_map(df, state)
new_heatmap(z=mean_map)

### Geo-plots

In [ ]:
from utils.plots import hist2d_to_geo

# get block infos for whole df
binned_data, bins = bin_data_frame(df,state)

counts_sum = binned_data.sum(0).sum(0)
counts_mean = binned_data.mean(0).mean(0)

blocks_geo, blocks_info = hist2d_to_geo(counts_mean,xbins,ybins,filter_zero=True)

sub = df.sample(5_000)

In [ ]:
mapbox_access_token = open(".mapbox_token").read()
# mapbox_access_token = "open-street-map"
mapbox_access_token

on_state = ('#f050ae', 8)
off_state = ('#ffab00', 8)

mapbox_styles = {
    "sat": "mapbox://styles/bernsblack/ckecz0wr52pfc1at7tvu43fmj", 
    "mono": "mapbox://styles/bernsblack/ckecyyizy065w19psrikmeo5d",
    "dark": "mapbox://styles/bernsblack/ckeikbchd254619s57np6iyum",
}

color_palette = ["#33a8c7","#52e3e1","#a0e426","#fdf148","#ffab00","#f77976","#f050ae","#d883ff","#9336fd"]

In [ ]:
heatmap_geo = go.Choroplethmapbox(
    geojson=blocks_geo,
    locations=blocks_info.id,
    z=blocks_info.value,
    colorscale="Viridis",
    zmin=blocks_info.value.min(),
    zmax=blocks_info.value.max(),
    marker_opacity=0.5,
    marker_line_width=0,
    name='heatmap',
    hoverinfo='text',
    hovertext=blocks_info[['y','x','value']],
)

# scatter_geo = go.Scattermapbox(
#     lon=sub.Longitude,
#     lat=sub.Latitude,
#     opacity=.4,
#     hoverinfo='skip',
#     name='scatter',
# )

scatter_geo = []
for cat in crime_categories:
    tmp = sub[sub["Primary Type"] == cat]
    scatter = go.Scattermapbox(
        lon=tmp.Longitude,
        lat=tmp.Latitude,
        opacity=.4,
        hoverinfo='skip',
        name=cat,
    )
    scatter_geo.append(scatter)
    
    

fig_geo = go.FigureWidget(data=[heatmap_geo, *scatter_geo])

fig_geo.update_layout(
    clickmode='event+select',
    mapbox_style=mapbox_styles["sat"],
    mapbox_accesstoken=mapbox_access_token,
    mapbox_zoom=10,
    mapbox_center = {"lat": np.mean(ybins), "lon": np.mean(xbins)},
    margin={"r":0,"t":30,"l":0,"b":10},
    height=500,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
    ),
)

def heatmap_geo_on_select(trace,points,selector):
    global state
    
    inds = points.point_inds
    selection = blocks_info.iloc[inds]
    _,_,f_x_min,f_y_min = selection.min()
    _,_,f_x_max,f_y_max = selection.max()

    state.lon_min, state.lon_max = xbins[[f_x_min,f_x_max]]
    state.lat_min, state.lat_max = ybins[[f_y_min,f_y_max]]
    
    
heatmap_geo = fig_geo.data[0]    
heatmap_geo.on_selection(heatmap_geo_on_select)

fig_geo

---
### Animated Crime Over Time

In [ ]:
def new_ffloor(delta):
    def res(value): 
        return ffloor(value, delta)
    return res



sub = df[['t', 'Longitude', 'Latitude','c', 'Primary Type', 'Date']].sample(10_000).copy()
sub['date_str'] = sub.Date.dt.floor(FREQ).dt.strftime('%c')
sub['count'] = np.ones_like(sub.t)
ff = new_ffloor(0.01)
sub[['Longitude', 'Latitude']] = sub[['Longitude', 'Latitude']].apply(ff)

gb = sub.groupby(by=['Longitude', 'Latitude','t','c'])
points = gb.first()
points['count'] = gb.count()['count']
# points['count'] = points['count'].apply(lambda x: np.log2(1+x))
points['count'] = points['count']/points['count'].min()
points = points.reset_index()
points = points.sort_values(['t','c'])

points = points[points.t < 4*5]
points

In [ ]:
import plotly.express as px

points['size'] = 10*points['count']

fig = px.scatter(
    data_frame=points, 
    x="Longitude", 
    y="Latitude", 
    title="Animated Crime Over Time",
#     facet_col="Primary Type",
#     facet_col_wrap=3,
    width=800, 
    height=800,
    opacity=.5,
    size="size",
    size_max=points['size'].max(),
    color='Primary Type',
#     marginal_x='violin',
    animation_frame="t",
#     animation_group="t",
#     color="count",
)

fig.update_layout(
    margin=dict(l=20, r=50, t=50, b=20),
    paper_bgcolor="LightSteelBlue",
#     transition ={'duration': 1},
#     frame={'duration': 100},
)

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 500
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 0

fig

---
## 2D Histogram

In [ ]:
# display distribution of a sample of the data
fig_height=1000
px.density_heatmap(
    data_frame=df.sample(10000),
    x="Longitude",
    y="Latitude", 
    marginal_x="histogram",
    marginal_y="histogram",
    height=fig_height,
    width=ratio_xy*fig_height,
    nbinsx=nx,
    nbinsy=ny,
)

---